<a href="https://colab.research.google.com/github/FeliciaStephen/EduVid/blob/main/video_generation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install edge_tts

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=2a409476cea7557c83fe9a26d36e51b5f3278d78377b70f10a7b0e6d0e7ddb71
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [ ]:
!pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=59eb3d847b0db26e6dac17dc982106e3b128d60d184ad41106a1ce2e4cb97fbd
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper


In [ ]:
import edge_tts

async def generate_audio(text,outputFilename):
    communicate = edge_tts.Communicate(text,"en-AU-WilliamNeural", rate="-20%")
    await communicate.save(outputFilename)







In [ ]:
!pip install whisper-timestamped


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.7/801.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s

In [ ]:
import whisper_timestamped as whisper
from whisper_timestamped import load_model, transcribe_timestamped
import re

def generate_timed_captions(audio_filename,model_size="base"):
    WHISPER_MODEL = load_model(model_size)

    gen = transcribe_timestamped(WHISPER_MODEL, audio_filename, verbose=False, fp16=False)

    return getCaptionsWithTime(gen)

def splitWordsBySize(words, maxCaptionSize):

    halfCaptionSize = maxCaptionSize / 2
    captions = []
    while words:
        caption = words[0]
        words = words[1:]
        while words and len(caption + ' ' + words[0]) <= maxCaptionSize:
            caption += ' ' + words[0]
            words = words[1:]
            if len(caption) >= halfCaptionSize and words:
                break
        captions.append(caption)
    return captions

def getTimestampMapping(whisper_analysis):

    index = 0
    locationToTimestamp = {}
    for segment in whisper_analysis['segments']:
        for word in segment['words']:
            newIndex = index + len(word['text'])+1
            locationToTimestamp[(index, newIndex)] = word['end']
            index = newIndex
    return locationToTimestamp

def cleanWord(word):

    return re.sub(r'[^\w\s\-_"\'\']', '', word)

def interpolateTimeFromDict(word_position, d):

    for key, value in d.items():
        if key[0] <= word_position <= key[1]:
            return value
    return None

def getCaptionsWithTime(whisper_analysis, maxCaptionSize=15, considerPunctuation=False):

    wordLocationToTime = getTimestampMapping(whisper_analysis)
    position = 0
    start_time = 0
    CaptionsPairs = []
    text = whisper_analysis['text']

    if considerPunctuation:
        sentences = re.split(r'(?<=[.!?]) +', text)
        words = [word for sentence in sentences for word in splitWordsBySize(sentence.split(), maxCaptionSize)]
    else:
        words = text.split()
        words = [cleanWord(word) for word in splitWordsBySize(words, maxCaptionSize)]

    for word in words:
        position += len(word) + 1
        end_time = interpolateTimeFromDict(position, wordLocationToTime)
        if end_time and word:
            CaptionsPairs.append(((start_time, end_time), word))
            start_time = end_time

    return CaptionsPairs


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [ ]:
import os
from datetime import datetime
import json

# Log types
LOG_TYPE_GPT = "GPT"
LOG_TYPE_PEXEL = "PEXEL"

# log directory paths
DIRECTORY_LOG_GPT = ".logs/gpt_logs"
DIRECTORY_LOG_PEXEL = ".logs/pexel_logs"

# method to log response from pexel and openai
def log_response(log_type, query,response):
    log_entry = {
        "query": query,
        "response": response,
        "timestamp": datetime.now().isoformat()
    }
    if log_type == LOG_TYPE_GPT:
        if not os.path.exists(DIRECTORY_LOG_GPT):
            os.makedirs(DIRECTORY_LOG_GPT)
        filename = '{}_gpt3.txt'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))
        filepath = os.path.join(DIRECTORY_LOG_GPT, filename)
        with open(filepath, "w") as outfile:
            outfile.write(json.dumps(log_entry) + '\n')

    if log_type == LOG_TYPE_PEXEL:
        if not os.path.exists(DIRECTORY_LOG_PEXEL):
            os.makedirs(DIRECTORY_LOG_PEXEL)
        filename = '{}_pexel.txt'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))
        filepath = os.path.join(DIRECTORY_LOG_PEXEL, filename)
        with open(filepath, "w") as outfile:
            outfile.write(json.dumps(log_entry) + '\n')


In [ ]:
import os
import requests
# from utility.utils import log_response,LOG_TYPE_PEXEL

PEXELS_API_KEY = "RnAJsbUZ1YNYN2KRk1WAYWq4zYVdVVadBPncdWbbrGFxXQeXu2l7ouc4"

def search_videos(query_string, orientation_landscape=True):


    url = "https://api.pexels.com/videos/search"
    headers = {
        "Authorization": PEXELS_API_KEY,
        "User-Agent": "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    params = {
        "query": query_string,
        "orientation": "landscape" if orientation_landscape else "portrait",
        "per_page": 15
    }

    # response = requests.get(url, headers=headers, params=params)
    # json_data = response.json()
    # log_response(LOG_TYPE_PEXEL,query_string,response.json())
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Will raise an error for HTTP failures
        json_data = response.json()
        print("Raw API Response:", json_data)  # Debugging

        # Debugging: Print API response
        print("Response Status Code:", response.status_code)
        print("Response JSON:", json_data)

        log_response(LOG_TYPE_PEXEL, query_string, json_data)  # Assuming log_response exists
        return json_data
    except requests.exceptions.RequestException as e:
        print("API Request Error:", e)
        return None

    return json_data


def getBestVideo(query_string, orientation_landscape=True, used_vids=[]):


    print("Calling search_videos...")

    vids = search_videos(query_string, orientation_landscape)
    # print("API Response:", vids)
    videos = vids['videos']  # Extract the videos list from JSON

    # Filter and extract videos with width and height as 1920x1080 for landscape or 1080x1920 for portrait
    if orientation_landscape:
        filtered_videos = [video for video in videos if video['width'] >= 1920 and video['height'] >= 1080 and video['width']/video['height'] == 16/9]
    else:
        filtered_videos = [video for video in videos if video['width'] >= 1080 and video['height'] >= 1920 and video['height']/video['width'] == 16/9]

    # Sort the filtered videos by duration in ascending order
    sorted_videos = sorted(filtered_videos, key=lambda x: abs(15-int(x['duration'])))

    # Extract the top 3 videos' URLs
    for video in sorted_videos:
        for video_file in video['video_files']:
            if orientation_landscape:
                if video_file['width'] == 1920 and video_file['height'] == 1080:
                    if not (video_file['link'].split('.hd')[0] in used_vids):
                        return video_file['link']
            else:
                if video_file['width'] == 1080 and video_file['height'] == 1920:
                    if not (video_file['link'].split('.hd')[0] in used_vids):
                        return video_file['link']
    print("NO LINKS found for this round of search with query :", query_string)
    return None


def generate_video_url(timed_video_searches,video_server):
        # timed_video_urls = []
        # if video_server == "pexel":
        #     used_links = []
        #     for (t1, t2), search_terms in timed_video_searches:
        #         url = ""
        #         for query in search_terms:

        #             url = getBestVideo(query, orientation_landscape=True, used_vids=used_links)
        #             if url:
        #                 used_links.append(url.split('.hd')[0])
        #                 break
        #         timed_video_urls.append([[t1, t2], url])
        # # elif video_server == "stable_diffusion":
        # #     timed_video_urls = get_images_for_video(timed_video_searches)

        # return timed_video_urls
        timed_video_urls = []

        if video_server == "pexel":
            used_links = []

            for (t1, t2), search_terms in timed_video_searches:
                url = None  # Use None instead of "" to differentiate between failed and successful searches

                for query in search_terms:
                    url = getBestVideo(query, orientation_landscape=True, used_vids=used_links)

                    if url:  # If a valid video is found, stop searching
                        used_links.append(url.split('.hd')[0])
                        break

            # Append only if a URL was found, otherwise use a default black screen
                if url:
                    timed_video_urls.append([[t1, t2], url])
                else:
                    print(f"Warning: No valid video found for {search_terms}. Using fallback.")
                    timed_video_urls.append([[t1, t2], None])  # Indicate missing video

        return timed_video_urls


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
import json
import re
from datetime import datetime
# from utility.utils import log_response,LOG_TYPE_GPT

# if len(os.environ.get("GROQ_API_KEY")) > 30:
from groq import Groq
model = "llama3-70b-8192"
client = Groq(
    api_key="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",
    )
# else:
#     model = "gpt-4o"
#     OPENAI_API_KEY = os.environ.get('OPENAI_KEY')
#     client = OpenAI(api_key=OPENAI_API_KEY)

log_directory = ".logs/gpt_logs"

prompt = """# Instructions

Given the following video script and timed captions, extract three visually concrete and specific keywords for each time segment that can be used to search for background videos. The keywords should be short and capture the main essence of the sentence. They can be synonyms or related terms. If a caption is vague or general, consider the next timed caption for more context. If a keyword is a single word, try to return a two-word keyword that is visually concrete. If a time frame contains two or more important pieces of information, divide it into shorter time frames with one keyword each. Ensure that the time periods are strictly consecutive and cover the entire length of the video. Each keyword should cover between 2-4 seconds. The output should be in JSON format, like this: [[[t1, t2], ["keyword1", "keyword2", "keyword3"]], [[t2, t3], ["keyword4", "keyword5", "keyword6"]], ...]. Please handle all edge cases, such as overlapping time segments, vague or general captions, and single-word keywords.

For example, if the caption is 'The cheetah is the fastest land animal, capable of running at speeds up to 75 mph', the keywords should include 'cheetah running', 'fastest animal', and '75 mph'. Similarly, for 'The Great Wall of China is one of the most iconic landmarks in the world', the keywords should be 'Great Wall of China', 'iconic landmark', and 'China landmark'.

Important Guidelines:

Use only English in your text queries.
Each search string must depict something visual.
The depictions have to be extremely visually concrete, like rainy street, or cat sleeping.
'emotional moment' <= BAD, because it doesn't depict something visually.
'crying child' <= GOOD, because it depicts something visual.
The list must always contain the most relevant and appropriate query searches.
['Car', 'Car driving', 'Car racing', 'Car parked'] <= BAD, because it's 4 strings.
['Fast car'] <= GOOD, because it's 1 string.
['Un chien', 'une voiture rapide', 'une maison rouge'] <= BAD, because the text query is NOT in English.

Note: Your response should be the response only and no extra text or data.
  """

# prompt = """
# # Instructions

# You will be given a video script with corresponding timed captions. Your task is to extract **exactly three visually descriptive and contextually accurate keywords** for each time segment. These keywords will be used to search for background videos. Ensure the keywords are:

# 1. **Visually specific**: They must depict something concrete that can be directly visualized, such as "red apple," "mountain sunrise," or "kids playing soccer."
# 2. **Contextually relevant**: Use the main essence of the sentence, considering adjacent captions for clarity if the meaning is vague or general.
# 3. **Optimally descriptive**: Avoid single-word keywords unless they are inherently descriptive (e.g., "volcano"). Prefer two- or three-word phrases like "erupting volcano" or "dense rainforest."
# 4. **Diverse**: Avoid repetitive keywords that describe the same concept within a short timeframe. For example, avoid multiple variations like ["car," "fast car," "red car"] in the same segment.

# ### Key Rules:
# - If a time segment contains multiple distinct pieces of information, divide it into smaller consecutive segments, ensuring complete and logical coverage of the video duration.
# - If a caption is vague or lacks visual detail, use the next caption for better context.
# - All time segments should cover 2-4 seconds and must strictly follow the video's timeline without overlaps or gaps.

# ### JSON Output Format:
# Return your output in the following format:
# [ [[t1, t2], ["keyword1", "keyword2", "keyword3"]], [[t2, t3], ["keyword4", "keyword5", "keyword6"]], ... ]

# ### Examples:
# 1. For the caption: "The cheetah is the fastest land animal, capable of running at speeds up to 75 mph":
#    Output: [[0, 4], ["cheetah running", "fastest animal", "75 mph"]]

# 2. For the caption: "The Great Wall of China is one of the most iconic landmarks in the world":
#    Output: [[0, 4], ["Great Wall of China", "iconic landmark", "China landmark"]]

# 3. For the captions:
#    - "A small child is playing with a colorful kite."
#    - "The child runs through a green field while the kite flies in the sky."
#    Output:
#    [
#      [[0, 3], ["child with kite", "colorful kite", "playing child"]],
#      [[3, 6], ["child running", "green field", "kite in sky"]]
#    ]

# ### Additional Guidelines:
# - Ensure the keywords are strictly in **English**.
# - Avoid abstract or non-visual phrases like "emotional moment" or "historical event." Instead, use terms like "crying child" or "ancient castle."
# - Keep the output concise, accurate, and strictly adhere to the JSON format.

#  """

# prompt = """
# # Instructions

# You will be given a video script with corresponding timed captions related to **science topics from grades 6-10**. Your task is to extract **exactly three visually descriptive and contextually accurate science-related keywords** for each time segment.

# ### Guidelines:
# 1. **Science-Relevant**: Ensure keywords are related to scientific concepts (e.g., "photosynthesis process", "magnetic field lines", "chemical reaction").
# 2. **Avoid Generic Terms**: Skip non-science phrases like "kids playing" or "nature scene".
# 3. **Use Domain Knowledge**: If a caption is vague, infer the scientific topic from adjacent captions.

# ### Examples:
# 1. Caption: "Plants use sunlight to make their own food."
#    ✅ Output: [[0, 4], ["photosynthesis", "sunlight absorption", "chlorophyll process"]]

# 2. Caption: "Iron filings align themselves along the lines of force."
#    ✅ Output: [[4, 8], ["magnetic field lines", "iron filings", "force field"]]

# 3. Caption: "Water evaporates faster in heat."
#    ✅ Output: [[8, 12], ["evaporation process", "water vapor", "heat energy"]]

# Ensure the output strictly follows this JSON format:
# [ [[t1, t2], ["keyword1", "keyword2", "keyword3"]], [[t2, t3], ["keyword4", "keyword5", "keyword6"]], ... ]

# """





def fix_json(json_str):
    # Replace typographical apostrophes with straight quotes
    json_str = json_str.replace("’", "'")
    # Replace any incorrect quotes (e.g., mixed single and double quotes)
    json_str = json_str.replace("“", "\"").replace("”", "\"").replace("‘", "\"").replace("’", "\"")
    # Add escaping for quotes within the strings
    json_str = json_str.replace('"you didn"t"', '"you didn\'t"')
    return json_str



import re

# def getVideoSearchQueriesTimed(script, captions_timed):
#     try:
#         # Call the OpenAI model to generate content
#         content = call_OpenAI(script, captions_timed)

#         # Parse the content into a list of tuples (time segment and search terms)
#         search_terms = []
#         sentences = content.split('], [')  # Split JSON-like structure into segments

#         for i, sentence in enumerate(sentences):
#             # Extract start and end time
#             start_time, end_time = captions_timed[i][0]

#             # Clean and format the extracted search terms
#             terms = re.findall(r'\"(.*?)\"', sentence)
#             search_terms.append(([start_time, end_time], terms))

#         return search_terms

#     except Exception as e:
#         print("Error getting video search queries:", e)
#         return None

def getVideoSearchQueriesTimed(script, captions_timed):
    try:
        content = call_OpenAI(script, captions_timed)

        search_terms = []
        sentences = content.split('], [')

        min_length = min(len(sentences), len(captions_timed))  # Ensure safe indexing

        for i in range(min_length):
            if not captions_timed[i] or len(captions_timed[i]) < 1:  # Prevent index error
                continue

            start_time, end_time = captions_timed[i][0]
            terms = re.findall(r'\"(.*?)\"', sentences[i])
            search_terms.append(([start_time, end_time], terms))

        return search_terms

    except Exception as e:
        print("Error getting video search queries:", e)
        return None


def call_OpenAI(script, captions_timed):
    user_content = """Script: {}
Timed Captions:{}
""".format(script, "".join(map(str, captions_timed)))

    print("Content", user_content)

    # Send request to OpenAI API
    response = client.chat.completions.create(
        model=model,
        temperature=1,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_content}
        ]
    )

    # Extract and clean up the response text
    text = response.choices[0].message.content.strip()
    text = re.sub('\\s+', ' ', text)

    print("Text", text)

    # Log the response if needed
    log_response(LOG_TYPE_GPT, script, text)

    return text
def merge_empty_intervals(segments):
    merged = []
    i = 0
    while i < len(segments):
        interval, url = segments[i]
        if url is None:
            # Find consecutive None intervals
            j = i + 1
            while j < len(segments) and segments[j][1] is None:
                j += 1

            # Merge consecutive None intervals with the previous valid URL
            if i > 0:
                prev_interval, prev_url = merged[-1]
                if prev_url is not None and prev_interval[1] == interval[0]:
                    merged[-1] = [[prev_interval[0], segments[j-1][0][1]], prev_url]
                else:
                    merged.append([interval, prev_url])
            else:
                merged.append([interval, None])

            i = j
        else:
            merged.append([interval, url])
            i += 1

    return merged


In [ ]:
!pip install openai

In [ ]:
!pip install asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install moviepy

In [ ]:
!apt-get install imagemagick


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nan

In [ ]:
!wget -q https://github.com/google/fonts/raw/main/apache/opensans/OpenSans%5Bwdth%2Cwght%5D.ttf -O OpenSans-Regular.ttf
!mv OpenSans-Regular.ttf /usr/share/fonts/truetype/
!fc-cache -f -v


/usr/share/fonts: caching, new cache contents: 0 fonts, 5 dirs
/usr/share/fonts/cMap: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/cmap: caching, new cache contents: 0 fonts, 5 dirs
/usr/share/fonts/cmap/adobe-cns1: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/cmap/adobe-gb1: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/cmap/adobe-japan1: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/cmap/adobe-japan2: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/cmap/adobe-korea1: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/opentype: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/opentype/urw-base35: caching, new cache contents: 35 fonts, 0 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 4 dirs
/usr/share/fonts/truetype/droid: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/f

In [ ]:
!pip install numpy

In [ ]:
!pip install --upgrade Pillow


In [ ]:
import os
import platform
import subprocess
import tempfile
import requests
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import (AudioFileClip, CompositeAudioClip, CompositeVideoClip, ImageClip, VideoFileClip)

def download_file(url, filename):
    """
    Downloads a file from the given URL to the specified filename.
    """
    with open(filename, 'wb') as f:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        f.write(response.content)

def search_program(program_name):
    """
    Searches for a program's executable path.
    """
    try:
        search_cmd = "where" if platform.system() == "Windows" else "which"
        return subprocess.check_output([search_cmd, program_name]).decode().strip()
    except subprocess.CalledProcessError:
        return None

import numpy as np

def get_output_media(audio_file_path, timed_captions, background_video_data, video_server):
    OUTPUT_FILE_NAME = "rendered_video.mp4"

    visual_clips = []
    fps = 25  # Set a default fps value for all video clips
    audio_clips = []
    audio_file_clip = AudioFileClip(audio_file_path)
    audio_clips.append(audio_file_clip)

    for (t1, t2), video_url in background_video_data:

        # Download the video file
        video_filename = tempfile.NamedTemporaryFile(delete=False).name
        download_file(video_url, video_filename)

        # Create VideoFileClip from the downloaded file
        video_clip = VideoFileClip(video_filename)
        video_clip = video_clip.set_start(t1)
        video_clip = video_clip.set_end(t2)
        # video_clip = video_clip.loop(duration=audio_file_clip.duration)

        # visual_clips.insert(0, video_clip)

        # Ensure the fps is set for each video clip
        video_clip = video_clip.set_fps(fps)
        if video_clip is None or not hasattr(video_clip, 'fps'):
              print("Failed to create video clip.")
              continue  # Skip this clip if it's invalid

        visual_clips.append(video_clip)



    # for (t1, t2), text in timed_captions:
    #     # Create text image using Pillow
    #     img = Image.new('RGBA', (1920, 1080), color=(0, 0, 0, 0))  # Fully transparent background

    #     d = ImageDraw.Draw(img)

    #     # font = ImageFont.load_default()
    #     # font = ImageFont.truetype("/Arial_Bold_Italic.ttf", 40)
    #     font_path = "/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf"  # Common font in Colab
    #     font = ImageFont.truetype(font_path, 80)

    #     # Calculate text size and position
    #     left, top, right, bottom = font.getbbox(text)
    #     text_width = right - left
    #     text_height = bottom - top
    #     text_x = (1920 - text_width) // 2
    #     text_y = 500


    #     d.text((text_x, text_y), text, font=font, fill=(255, 255, 255, 255), stroke_width=3, stroke_fill=(0, 0, 0, 255))
    #     img.save("debug_text.png")

    #     # Convert Pillow image to NumPy array and then to MoviePy ImageClip
    #     img_array = np.array(img)
    #     text_clip = ImageClip(img_array, ismask=False).set_start(t1).set_end(t2).set_position(("center", "center")).set_fps(25)

    #     text_clip = text_clip.set_start(t1)
    #     text_clip = text_clip.set_end(t2)
    #     text_clip = text_clip.set_position(["center", 800])
    #     text_clip = text_clip.set_fps(25)  # Ensure FPS for the text clips as well
    #     visual_clips.append(text_clip)
    for (t1, t2), text in timed_captions:
      # Create a blank image for the text
      img = Image.new('RGBA', (1920, 1080), color=(0, 0, 0, 0))  # Transparent background
      d = ImageDraw.Draw(img)

      # Load the font
      font_path = "/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf"  # Common font in Colab
      font = ImageFont.truetype(font_path, 80)

      # Calculate text size and position
      left, top, right, bottom = font.getbbox(text)
      text_width = right - left
      text_height = bottom - top
      text_x = (1920 - text_width) // 2
      text_y = 800  # Adjust the y-position as needed

      # Draw the text with a stroke
      d.text((text_x, text_y), text, font=font, fill=(255, 255, 255, 255), stroke_width=3, stroke_fill=(0, 0, 0, 255))

      # Save for debugging (optional)
      # img.save(f"debug_text_{t1}.png")

      # Convert Pillow image to a NumPy array and then to a MoviePy ImageClip
      img_array = np.array(img)
      text_clip = (
          ImageClip(img_array, ismask=False)
          .set_start(t1)
          .set_end(t2)
          .set_position(("center", "center"))
          .set_fps(fps)
      )

      # Add the text clip to the visual clips
      visual_clips.append(text_clip)

    visual_clips = [clip for clip in visual_clips if clip.duration > 0]
    for clip in visual_clips:
        clip = clip.set_fps(25)  # Ensure fps is set consistently


    video = CompositeVideoClip(visual_clips)
    video.fps=25

    if audio_clips:
        audio = CompositeAudioClip(audio_clips)
        video.duration = audio.duration
        video.audio = audio
    print("Visual clips length:", len(visual_clips))
    print("Audio clips length:", len(audio_clips))
    if not audio_clips:
      print("No audio clips were created.")
    if not visual_clips:
      print("No visual clips were created.")
    print("Composite Video Duration:", video.duration)
    print("Composite Video FPS:", video.fps)
    # for idx, clip in enumerate(visual_clips):
    #     print(f"Clip {idx} Duration:", clip.duration)
    #     print(f"Clip {idx} FPS:", getattr(clip, 'fps', 'Not Set'))




    video.write_videofile(OUTPUT_FILE_NAME, codec='libx264', audio_codec='aac', fps=fps, preset='veryfast')

    # Clean up downloaded files
    for (t1, t2), video_url in background_video_data:
        video_filename = tempfile.NamedTemporaryFile(delete=False).name
        os.remove(video_filename)

    return OUTPUT_FILE_NAME




  if event.key is 'enter':



In [ ]:
import os
import platform
import subprocess
import tempfile
import requests
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import (AudioFileClip, CompositeAudioClip, CompositeVideoClip, ImageClip, VideoFileClip, ColorClip)

def download_file(url, filename):
    """
    Downloads a file from the given URL to the specified filename.
    """
    try:
        with open(filename, 'wb') as f:
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            f.write(response.content)
        return filename
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return None

def get_output_media(audio_file_path, timed_captions, background_video_data, video_server):
    OUTPUT_FILE_NAME = "rendered_video.mp4"
    visual_clips = []
    fps = 25  # Default FPS
    audio_clips = []
    audio_file_clip = AudioFileClip(audio_file_path)
    audio_clips.append(audio_file_clip)

    for (t1, t2), video_url in background_video_data:
        if not video_url:
            print(f"No video URL for time range {t1}-{t2}, using a blank background.")
            blank_clip = ColorClip(size=(1920, 1080), color=(0, 0, 0), duration=t2 - t1)
            blank_clip = blank_clip.set_start(t1).set_end(t2).set_fps(fps)
            visual_clips.append(blank_clip)
            continue

        video_filename = tempfile.NamedTemporaryFile(delete=False).name
        video_filename = download_file(video_url, video_filename)
        if not video_filename:
            print(f"Skipping video segment {t1}-{t2} due to download failure.")
            continue

        video_clip = VideoFileClip(video_filename).subclip(0, min(t2 - t1, VideoFileClip(video_filename).duration))
        video_clip = video_clip.set_start(t1).set_end(t2).set_fps(fps)
        visual_clips.append(video_clip)

    for (t1, t2), text in timed_captions:
        img = Image.new('RGBA', (1920, 1080), color=(0, 0, 0, 0))
        d = ImageDraw.Draw(img)
        font_path = "/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf"
        font = ImageFont.truetype(font_path, 80)
        left, top, right, bottom = font.getbbox(text)
        text_width = right - left
        text_x = (1920 - text_width) // 2
        text_y = 800
        d.text((text_x, text_y), text, font=font, fill=(255, 255, 255, 255), stroke_width=3, stroke_fill=(0, 0, 0, 255))
        img_array = np.array(img)
        text_clip = ImageClip(img_array, ismask=False).set_start(t1).set_end(t2).set_position("center").set_fps(fps)
        visual_clips.append(text_clip)

    if not visual_clips:
        print("No visual clips available. Creating a default black screen.")
        visual_clips.append(ColorClip(size=(1920, 1080), color=(0, 0, 0), duration=audio_file_clip.duration).set_fps(fps))

    video = CompositeVideoClip(visual_clips)
    video.fps = fps

    if audio_clips:
        video.audio = CompositeAudioClip(audio_clips)

    print("Rendering final video...")
    video.write_videofile(OUTPUT_FILE_NAME, codec='libx264', audio_codec='aac', fps=fps, preset='veryfast')

    return OUTPUT_FILE_NAME


In [ ]:
# # transitions
# import os
# import platform
# import subprocess
# import tempfile
# import requests
# import numpy as np
# from PIL import Image, ImageDraw, ImageFont
# from moviepy.editor import (AudioFileClip, CompositeAudioClip, CompositeVideoClip, ImageClip, VideoFileClip)

# def download_file(url, filename):
#     """
#     Downloads a file from the given URL to the specified filename.
#     """
#     with open(filename, 'wb') as f:
#         headers = {
#             "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#         }
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()  # Raise an error for bad status codes
#         f.write(response.content)

# def search_program(program_name):
#     """
#     Searches for a program's executable path.
#     """
#     try:
#         search_cmd = "where" if platform.system() == "Windows" else "which"
#         return subprocess.check_output([search_cmd, program_name]).decode().strip()
#     except subprocess.CalledProcessError:
#         return None

# def get_output_media(audio_file_path, timed_captions, background_video_data, video_server):
#     OUTPUT_FILE_NAME = "rendered_video.mp4"

#     visual_clips = []
#     fps = 25  # Set a default fps value for all video clips
#     audio_clips = []
#     audio_file_clip = AudioFileClip(audio_file_path)
#     audio_clips.append(audio_file_clip)

#     for (t1, t2), video_url in background_video_data:
#         # Download the video file
#         video_filename = tempfile.NamedTemporaryFile(delete=False).name
#         download_file(video_url, video_filename)

#         # Create VideoFileClip from the downloaded file
#         video_clip = VideoFileClip(video_filename)
#         video_clip = video_clip.set_start(t1)
#         video_clip = video_clip.set_end(t2)
#         video_clip = video_clip.set_fps(fps)

#         if video_clip is None or not hasattr(video_clip, 'fps'):
#             print("Failed to create video clip.")
#             continue  # Skip this clip if it's invalid

#         visual_clips.append(video_clip)

#     # Add transitions between clips
#     transition_duration = 1  # 1 second transition between clips
#     visual_clips_with_transitions = []

#     for i in range(len(visual_clips) - 1):
#         current_clip = visual_clips[i]
#         next_clip = visual_clips[i + 1]

#         # Apply a crossfade transition between the current and next clip
#         current_clip = current_clip.crossfadeout(transition_duration)
#         # next_clip = next_clip.crossfadein(transition_duration)

#         # Append both clips with the transition effect
#         visual_clips_with_transitions.append(current_clip)
#         # visual_clips_with_transitions.append(next_clip)

#     # Ensure the last clip doesn't have a crossfadeout
#     visual_clips_with_transitions.append(visual_clips[-1])

#     # Add timed text captions as per your original code
#     for (t1, t2), text in timed_captions:
#         # Create a blank image for the text
#         img = Image.new('RGBA', (1920, 1080), color=(0, 0, 0, 0))  # Transparent background
#         d = ImageDraw.Draw(img)

#         # Load the font
#         font_path = "/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf"  # Common font in Colab
#         font = ImageFont.truetype(font_path, 80)

#         # Calculate text size and position
#         left, top, right, bottom = font.getbbox(text)
#         text_width = right - left
#         text_height = bottom - top
#         text_x = (1920 - text_width) // 2
#         text_y = 800  # Adjust the y-position as needed

#         # Draw the text with a stroke
#         d.text((text_x, text_y), text, font=font, fill=(255, 255, 255, 255), stroke_width=3, stroke_fill=(0, 0, 0, 255))

#         # Convert the image to a NumPy array and then to a MoviePy ImageClip
#         img_array = np.array(img)
#         text_clip = (
#             ImageClip(img_array, ismask=False)
#             .set_start(t1)
#             .set_end(t2)
#             .set_position(("center", "center"))
#             .set_fps(fps)
#         )

#         visual_clips_with_transitions.append(text_clip)

#     # Filter out any clips with zero duration
#     visual_clips_with_transitions = [clip for clip in visual_clips_with_transitions if clip.duration > 0]

#     # Create the final composite video
#     video = CompositeVideoClip(visual_clips_with_transitions)
#     video.fps = 25

#     if audio_clips:
#         audio = CompositeAudioClip(audio_clips)
#         video.duration = audio.duration
#         video.audio = audio

#     print("Visual clips length:", len(visual_clips_with_transitions))
#     print("Audio clips length:", len(audio_clips))
#     if not audio_clips:
#         print("No audio clips were created.")
#     if not visual_clips_with_transitions:
#         print("No visual clips were created.")
#     print("Composite Video Duration:", video.duration)
#     print("Composite Video FPS:", video.fps)

#     video.write_videofile(OUTPUT_FILE_NAME, codec='libx264', audio_codec='aac', fps=fps, preset='veryfast')

#     # Clean up downloaded files
#     for (t1, t2), video_url in background_video_data:
#         video_filename = tempfile.NamedTemporaryFile(delete=False).name
#         os.remove(video_filename)

#     return OUTPUT_FILE_NAME


In [ ]:
# parser = argparse.ArgumentParser(description="Generate a video from a topic.")
# parser.add_argument("topic", type=str, help="The topic for the video")
model = "llama3-70b-8192"
client = Groq(
    api_key="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",
)
# args = parser.parse_args()
# SAMPLE_TOPIC = args.topic
import asyncio
SAMPLE_FILE_NAME = "audio_tts.wav"
VIDEO_SERVER = "pexel"
# response = "Photosynthesis is how plants make food from sunlight. They take in carbon dioxide and water, using energy from sunlight. Chlorophyll, a green pigment, helps absorb this energy. Plants release oxygen as a byproduct, nourishing the air. This process happens in leaf cells, producing glucose, which fuels plant growth."

# response = "Rainwater harvesting collects and stores rainwater from rooftops in tanks for future use, reducing water scarcity and stormwater runoff. The stored water can be used for watering plants, washing, flushing toilets, and irrigating gardens and farms, providing a sustainable and eco-friendly alternative to municipal water supplies."

# response = "Newton's Second Law of Motion says that how fast something moves depends on how heavy it is and how hard you push it. Imagine you have a small toy car and a big box. If you push both with the same strength, the toy car will move faster because it’s lighter. But the big box will move more slowly because it’s heavier. If you want the big box to move as fast as the toy car, you’ll have to push it much harder. So, the law is really about how things move when you push or pull them, and it helps us understand that heavier things need stronger pushes to move quickly."

# response = "This chapter discusses the significance of food and nutrients in maintaining good health. It highlights the variety of meals people typically consume, which often include a combination of grains, vegetables, and proteins. The chapter encourages students to explore the nutrients in different food items by conducting simple tests to identify carbohydrates, proteins, and fats. The chapter explains the major nutrients in food, including carbohydrates, proteins, fats, vitamins, and minerals, and their specific functions in the body. Carbohydrates and fats provide energy, while proteins support growth and repair. Vitamins protect against diseases, and minerals are essential for healthy growth. Additionally, dietary fibers and water play a crucial role in digestion and nutrient absorption. A balanced diet that includes all necessary nutrients in the right amounts is essential for maintaining health. The chapter also emphasizes that individual dietary needs vary depending on factors such as physical activity levels and age. It stresses the importance of cooking food properly to preserve nutrients, as improper cooking can lead to the loss of certain vitamins and minerals. Furthermore, the chapter addresses deficiency diseases that occur when the body lacks certain nutrients over extended periods. These diseases can be prevented by consuming a balanced diet that includes all essential nutrients, highlighting the importance of a well-planned diet in maintaining overall health."

# response = generate_script(SAMPLE_TOPIC)

response ="The water cycle moves water around the Earth. Water evaporates from oceans, forms clouds, and falls as rain or snow. This keeps lakes, rivers, and plants full of water."


await generate_audio(response, SAMPLE_FILE_NAME)


timed_captions = generate_timed_captions(SAMPLE_FILE_NAME)
print(timed_captions)

search_terms = getVideoSearchQueriesTimed(response, timed_captions)
print(search_terms)

background_video_urls = []
if search_terms is not None:
    background_video_urls = generate_video_url(search_terms, VIDEO_SERVER)
    print(background_video_urls)
else:
    print("No background video")
print("background_video_urls:", background_video_urls)


background_video_urls = merge_empty_intervals(background_video_urls)
print("background_video_urls:", background_video_urls)
if background_video_urls is not None:

    video = get_output_media(SAMPLE_FILE_NAME, timed_captions, background_video_urls, VIDEO_SERVER)
    print(video)
else:
    print("No video")


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 138MiB/s]


Detected language: English


100%|██████████| 1461/1461 [00:01<00:00, 1068.13frames/s]


[((0, np.float64(0.66)), 'The water'), ((np.float64(0.66), np.float64(1.44)), 'cycle moves'), ((np.float64(1.44), np.float64(2.24)), 'water around'), ((np.float64(2.24), np.float64(2.88)), 'the earth'), ((np.float64(2.88), np.float64(4.38)), 'Water'), ((np.float64(4.38), np.float64(5.3)), 'evaporates from'), ((np.float64(5.3), np.float64(6.46)), 'oceans forms'), ((np.float64(6.46), np.float64(7.34)), 'clouds and'), ((np.float64(7.34), np.float64(8.04)), 'falls as'), ((np.float64(8.04), np.float64(8.98)), 'rain or snow'), ((np.float64(8.98), np.float64(10.92)), 'This keeps'), ((np.float64(10.92), np.float64(11.8)), 'lakes rivers'), ((np.float64(11.8), np.float64(12.58)), 'and plants'), ((np.float64(12.58), np.float64(13.4)), 'full of water')]
Content Script: The water cycle moves water around the Earth. Water evaporates from oceans, forms clouds, and falls as rain or snow. This keeps lakes, rivers, and plants full of water.
Timed Captions:((0, np.float64(0.66)), 'The water')((np.float64

MoviePy - Done.
Moviepy - Writing video rendered_video.mp4



Moviepy - Done !
Moviepy - video ready rendered_video.mp4
rendered_video.mp4


In [ ]:
!pip install --upgrade Pillow


In [ ]:
import os
print(os.path.exists("/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf"))


True


In [ ]:
!git clone https://github.com/FeliciaStephen/EduVid.git

Cloning into 'EduVid'...


In [ ]:
%cd EduVid

/content/EduVid
